In [1]:
# coding:utf-8
import sys
sys.path.append('/usr/local/lib/python3.5/dist-packages')
import tensorflow as tf
import gym, time, random, threading
from gym import wrappers  # gymの画像保存
from keras.models import *
from keras.layers import *
from keras.utils import plot_model
from keras import backend as K
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'    # TensorFlow高速化用のワーニングを表示させない


Using TensorFlow backend.


In [2]:
# -- constants of Game
ENV = 'CartPole-v0'
env = gym.make(ENV)
NUM_STATES = env.observation_space.shape[0]     # CartPoleは4状態
NUM_ACTIONS = env.action_space.n        # CartPoleは、右に左に押す2アクション
NONE_STATE = np.zeros(NUM_STATES)

# -- constants of LocalBrain
MIN_BATCH = 5
LOSS_V = .5  # v loss coefficient
LOSS_ENTROPY = .01  # entropy coefficient
LEARNING_RATE = 5e-3
RMSPropDecaly = 0.99

# -- params of Advantage-ベルマン方程式
GAMMA = 0.99
N_STEP_RETURN = 5
GAMMA_N = GAMMA ** N_STEP_RETURN

N_WORKERS = 8   # スレッドの数
Tmax = 10   # 各スレッドの更新ステップ間隔

# ε-greedyのパラメータ
EPS_START = 0.5
EPS_END = 0.0
EPS_STEPS = 200*N_WORKERS

WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.


In [3]:
# --グローバルなTensorFlowのDeep Neural Networkのクラスです　-------
class ParameterServer:
    def __init__(self):
        with tf.variable_scope("parameter_server"):      # スレッド名で重み変数に名前を与え、識別します（Name Space）
            self.model = self._build_model()            # ニューラルネットワークの形を決定

        # serverのパラメータを宣言
        self.weights_params = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope="parameter_server")
        self.optimizer = tf.train.RMSPropOptimizer(LEARNING_RATE, RMSPropDecaly)    # loss関数を最小化していくoptimizerの定義です

    # 関数名がアンダースコア2つから始まるものは「外部から参照されない関数」、「1つは基本的に参照しない関数」という意味
    def _build_model(self):     # Kerasでネットワークの形を定義します
        l_input = Input(batch_shape=(None, NUM_STATES))
        l_dense = Dense(16, activation='relu')(l_input)
        out_actions = Dense(NUM_ACTIONS, activation='softmax')(l_dense)
        out_value = Dense(1, activation='linear')(l_dense)
        model = Model(inputs=[l_input], outputs=[out_actions, out_value])
        plot_model(model, to_file='A3C.png', show_shapes=True)  # Qネットワークの可視化
        return model


In [4]:
# --各スレッドで走るTensorFlowのDeep Neural Networkのクラスです　-------
class LocalBrain:
    def __init__(self, name, parameter_server):   # globalなparameter_serverをメンバ変数として持つ
        with tf.name_scope(name):
            self.train_queue = [[], [], [], [], []]  # s, a, r, s', s' terminal mask
            K.set_session(SESS)
            self.model = self._build_model()  # ニューラルネットワークの形を決定
            self._build_graph(name, parameter_server)  # ネットワークの学習やメソッドを定義

    def _build_model(self):     # Kerasでネットワークの形を定義します
        l_input = Input(batch_shape=(None, NUM_STATES))
        l_dense = Dense(16, activation='relu')(l_input)
        out_actions = Dense(NUM_ACTIONS, activation='softmax')(l_dense)
        out_value = Dense(1, activation='linear')(l_dense)
        model = Model(inputs=[l_input], outputs=[out_actions, out_value])
        model._make_predict_function()  # have to initialize before threading
        return model

    def _build_graph(self, name, parameter_server):      # TensorFlowでネットワークの重みをどう学習させるのかを定義します
        self.s_t = tf.placeholder(tf.float32, shape=(None, NUM_STATES))  # placeholderは変数が格納される予定地となります
        self.a_t = tf.placeholder(tf.float32, shape=(None, NUM_ACTIONS))
        self.r_t = tf.placeholder(tf.float32, shape=(None, 1))  # not immediate, but discounted n step reward

        p, v = self.model(self.s_t)

        # loss関数を定義します
        log_prob = tf.log(tf.reduce_sum(p * self.a_t, axis=1, keep_dims=True) + 1e-10)
        advantage = self.r_t - v
        loss_policy = - log_prob * tf.stop_gradient(advantage)  # stop_gradientでadvantageは定数として扱います
        loss_value = LOSS_V * tf.square(advantage)  # minimize value error
        entropy = LOSS_ENTROPY * tf.reduce_sum(p * tf.log(p + 1e-10), axis=1, keep_dims=True)  # maximize entropy (regularization)
        self.loss_total = tf.reduce_mean(loss_policy + loss_value + entropy)

        # 重みの変数を定義
        self.weights_params = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope=name)  # パラメータを宣言
        # 勾配を取得する定義
        self.grads = tf.gradients(self.loss_total, self.weights_params)

        # ParameterServerの重み変数を更新する定義(zipで各変数ごとに計算)
        self.update_global_weight_params = \
            parameter_server.optimizer.apply_gradients(zip(self.grads, parameter_server.weights_params))

        # PrameterServerの重み変数の値を、localBrainにコピーする定義
        self.pull_global_weight_params = [l_p.assign(g_p)
                                          for l_p, g_p in zip(self.weights_params, parameter_server.weights_params)]

        # localBrainの重み変数の値を、PrameterServerにコピーする定義
        self.push_local_weight_params = [g_p.assign(l_p)
                                          for g_p, l_p in zip(parameter_server.weights_params, self.weights_params)]

    def pull_parameter_server(self):  # localスレッドがglobalの重みを取得する
        SESS.run(self.pull_global_weight_params)

    def push_parameter_server(self):  # localスレッドの重みをglobalにコピーする
        SESS.run(self.push_local_weight_params)

    def update_parameter_server(self):     # localbrainの勾配でParameterServerの重みを学習・更新します
        if len(self.train_queue[0]) < MIN_BATCH:    # データがたまっていない場合は更新しない
            return

        s, a, r, s_, s_mask = self.train_queue
        self.train_queue = [[], [], [], [], []]
        s = np.vstack(s)    # vstackはvertical-stackで縦方向に行列を連結、いまはただのベクトル転置操作
        a = np.vstack(a)
        r = np.vstack(r)
        s_ = np.vstack(s_)
        s_mask = np.vstack(s_mask)

        # Nステップあとの状態s_から、その先得られるであろう時間割引総報酬vを求めます
        _, v = self.model.predict(s_)

        # N-1ステップあとまでの時間割引総報酬rに、Nから先に得られるであろう総報酬vに割引N乗したものを足します
        r = r + GAMMA_N * v * s_mask  # set v to 0 where s_ is terminal state
        feed_dict = {self.s_t: s, self.a_t: a, self.r_t: r}     # 重みの更新に使用するデータ
        SESS.run(self.update_global_weight_params, feed_dict)   # ParameterServerの重みを更新

    def predict_p(self, s):    # 状態sから各actionの確率pベクトルを返します
        p, v = self.model.predict(s)
        return p

    def train_push(self, s, a, r, s_):
        self.train_queue[0].append(s)
        self.train_queue[1].append(a)
        self.train_queue[2].append(r)

        if s_ is None:
            self.train_queue[3].append(NONE_STATE)
            self.train_queue[4].append(0.)
        else:
            self.train_queue[3].append(s_)
            self.train_queue[4].append(1.)

In [5]:
# --行動を決定するクラスです、CartPoleであれば、棒付き台車そのものになります　-------
class Agent:
    def __init__(self, name, parameter_server):
        self.brain = LocalBrain(name, parameter_server)   # 行動を決定するための脳（ニューラルネットワーク）
        self.memory = []        # s,a,r,s_の保存メモリ、　used for n_step return
        self.R = 0.             # 時間割引した、「いまからNステップ分あとまで」の総報酬R

    def act(self, s):
        if frames >= EPS_STEPS:   # ε-greedy法で行動を決定します 171115修正
            eps = EPS_END
        else:
            eps = EPS_START + frames * (EPS_END - EPS_START) / EPS_STEPS  # linearly interpolate

        if random.random() < eps:
            return random.randint(0, NUM_ACTIONS - 1)   # ランダムに行動
        else:
            s = np.array([s])
            p = self.brain.predict_p(s)

            # a = np.argmax(p)  # これだと確率最大の行動を、毎回選択

            a = np.random.choice(NUM_ACTIONS, p=p[0])
            # probability = p のこのコードだと、確率p[0]にしたがって、行動を選択
            # pにはいろいろな情報が入っていますが確率のベクトルは要素0番目
            return a

    def advantage_push_local_brain(self, s, a, r, s_):   # advantageを考慮したs,a,r,s_をbrainに与える
        def get_sample(memory, n):  # advantageを考慮し、メモリからnステップ後の状態とnステップ後までのRを取得する関数
            s, a, _, _ = memory[0]
            _, _, _, s_ = memory[n - 1]
            return s, a, self.R, s_

        # one-hotコーディングにしたa_catsをつくり、、s,a_cats,r,s_を自分のメモリに追加
        a_cats = np.zeros(NUM_ACTIONS)  # turn action into one-hot representation
        a_cats[a] = 1
        self.memory.append((s, a_cats, r, s_))

        # 前ステップの「時間割引Nステップ分の総報酬R」を使用して、現ステップのRを計算
        self.R = (self.R + r * GAMMA_N) / GAMMA     # r0はあとで引き算している、この式はヤロミルさんのサイトを参照

        # advantageを考慮しながら、LocalBrainに経験を入力する
        if s_ is None:
            while len(self.memory) > 0:
                n = len(self.memory)
                s, a, r, s_ = get_sample(self.memory, n)
                self.brain.train_push(s, a, r, s_)
                self.R = (self.R - self.memory[0][2]) / GAMMA
                self.memory.pop(0)

            self.R = 0  # 次の試行に向けて0にしておく

        if len(self.memory) >= N_STEP_RETURN:
            s, a, r, s_ = get_sample(self.memory, N_STEP_RETURN)
            self.brain.train_push(s, a, r, s_)
            self.R = self.R - self.memory[0][2]     # # r0を引き算
            self.memory.pop(0)

In [6]:
# --CartPoleを実行する環境です、TensorFlowのスレッドになります　-------
class Environment:
    total_reward_vec = np.zeros(10)  # 総報酬を10試行分格納して、平均総報酬をもとめる
    count_trial_each_thread = 0     # 各環境の試行数

    def __init__(self, name, thread_type, parameter_server):
        self.name = name
        self.thread_type = thread_type
        self.env = gym.make(ENV)
        self.agent = Agent(name, parameter_server)    # 環境内で行動するagentを生成

    def run(self):
        self.agent.brain.pull_parameter_server()  # ParameterSeverの重みを自身のLocalBrainにコピー
        global frames  # セッション全体での試行数、global変数を書き換える場合は、関数内でglobal宣言が必要です
        global isLearned

        if (self.thread_type is 'test') and (self.count_trial_each_thread == 0):
            self.env.reset()
            self.env = gym.wrappers.Monitor(self.env, './movie/A3C')  # 動画保存する場合

        s = self.env.reset()
        R = 0
        step = 0
        while True:
            if self.thread_type is 'test':
                self.env.render()   # 学習後のテストでは描画する
                time.sleep(0.1)

            a = self.agent.act(s)   # 行動を決定
            s_, r, done, info = self.env.step(a)   # 行動を実施
            step += 1
            frames += 1     # セッショントータルの行動回数をひとつ増やします

            r = 0
            if done:  # terminal state
                s_ = None
                if step < 199:
                    r = -1
                else:
                    r = 1

            # Advantageを考慮した報酬と経験を、localBrainにプッシュ
            self.agent.advantage_push_local_brain(s, a, r, s_)

            s = s_
            R += r
            if done or (step % Tmax == 0):  # 終了時がTmaxごとに、parameterServerの重みを更新し、それをコピーする
                if not(isLearned) and self.thread_type is 'learning':
                    self.agent.brain.update_parameter_server()
                    self.agent.brain.pull_parameter_server()

            if done:
                self.total_reward_vec = np.hstack((self.total_reward_vec[1:], step))  # トータル報酬の古いのを破棄して最新10個を保持
                self.count_trial_each_thread += 1  # このスレッドの総試行回数を増やす
                break
        # 総試行数、スレッド名、今回の報酬を出力
        print("スレッド："+self.name + "、試行数："+str(self.count_trial_each_thread) + "、今回のステップ:" + str(step)+"、平均ステップ："+str(self.total_reward_vec.mean()))

        # スレッドで平均報酬が一定を越えたら終了
        if self.total_reward_vec.mean() > 199:
            isLearned = True
            time.sleep(2.0)     # この間に他のlearningスレッドが止まります
            self.agent.brain.push_parameter_server()    # この成功したスレッドのパラメータをparameter-serverに渡します

In [7]:
# --スレッドになるクラスです　-------
class Worker_thread:
    # スレッドは学習環境environmentを持ちます
    def __init__(self, thread_name, thread_type, parameter_server):
        self.environment = Environment(thread_name, thread_type, parameter_server)
        self.thread_type = thread_type

    def run(self):
        while True:
            if not(isLearned) and self.thread_type is 'learning':     # learning threadが走る
                self.environment.run()

            if not(isLearned) and self.thread_type is 'test':    # test threadを止めておく
                time.sleep(1.0)

            if isLearned and self.thread_type is 'learning':     # learning threadを止めておく
                time.sleep(3.0)

            if isLearned and self.thread_type is 'test':     # test threadが走る
                time.sleep(3.0)
                self.environment.run()
                

In [8]:
# -- main ここからメイン関数です------------------------------
# M0.global変数の定義と、セッションの開始です
frames = 0              # 全スレッドで共有して使用する総ステップ数
isLearned = False       # 学習が終了したことを示すフラグ
SESS = tf.Session()     # TensorFlowのセッション開始

# M1.スレッドを作成します
with tf.device("/cpu:0"):
    parameter_server = ParameterServer()    # 全スレッドで共有するパラメータを持つエンティティです
    threads = []     # 並列して走るスレッド
    # 学習するスレッドを用意
    for i in range(N_WORKERS):
        thread_name = "local_thread"+str(i+1)
        threads.append(Worker_thread(thread_name=thread_name, thread_type="learning", parameter_server=parameter_server))

    # 学習後にテストで走るスレッドを用意
    threads.append(Worker_thread(thread_name="test_thread", thread_type="test", parameter_server=parameter_server))

# M2.TensorFlowでマルチスレッドを実行します
COORD = tf.train.Coordinator()                  # TensorFlowでマルチスレッドにするための準備です
SESS.run(tf.global_variables_initializer())     # TensorFlowを使う場合、最初に変数初期化をして、実行します

running_threads = []
for worker in threads:
    job = lambda: worker.run()      # この辺は、マルチスレッドを走らせる作法だと思って良い
    t = threading.Thread(target=job)
    t.start()
    #running_threads.append(t)

# M3.スレッドの終了を合わせます
#COORD.join(running_threads)

WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
Instructions for updating:
keep_dims is deprecated, use keepdims instead
WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
スレッド：local_thread6、試行数：1、今回のステップ:10、

スレッド：local_thread3、試行数：18、今回のステップ:11、平均ステップ：20.7
スレッド：local_thread2、試行数：19、今回のステップ:26、平均ステップ：18.7
スレッド：local_thread4、試行数：21、今回のステップ:47、平均ステップ：21.2
スレッド：local_thread6、試行数：21、今回のステップ:26、平均ステップ：21.2
スレッド：local_thread7、試行数：21、今回のステップ:23、平均ステップ：17.4
スレッド：local_thread8、試行数：22、今回のステップ:24、平均ステップ：20.7
スレッド：local_thread1、試行数：22、今回のステップ:26、平均ステップ：16.9
スレッド：local_thread3、試行数：19、今回のステップ:11、平均ステップ：17.9
スレッド：local_thread5、試行数：16、今回のステップ:17、平均ステップ：28.1
スレッド：local_thread8、試行数：23、今回のステップ:13、平均ステップ：20.8
スレッド：local_thread6、試行数：22、今回のステップ:15、平均ステップ：20.2
スレッド：local_thread1、試行数：23、今回のステップ:18、平均ステップ：17.3
スレッド：local_thread4、試行数：22、今回のステップ:24、平均ステップ：21.0
スレッド：local_thread6、試行数：23、今回のステップ:10、平均ステップ：19.4
スレッド：local_thread3、試行数：20、今回のステップ:14、平均ステップ：18.4
スレッド：local_thread2、試行数：20、今回のステップ:32、平均ステップ：20.0
スレッド：local_thread2、試行数：21、今回のステップ:9、平均ステップ：19.9
スレッド：local_thread5、試行数：17、今回のステップ:27、平均ステップ：28.7
スレッド：local_thread7、試行数：22、今回のステップ:42、平均ステップ：20.1
スレッド：local_thread4、試行数：23、今回のステップ:20、平均ステップ：18.5
スレッド：local_thread8、試行

スレッド：local_thread5、試行数：37、今回のステップ:36、平均ステップ：42.1
スレッド：local_thread7、試行数：45、今回のステップ:106、平均ステップ：40.3
スレッド：local_thread6、試行数：42、今回のステップ:23、平均ステップ：47.4
スレッド：local_thread3、試行数：41、今回のステップ:44、平均ステップ：31.8
スレッド：local_thread2、試行数：40、今回のステップ:37、平均ステップ：37.8
スレッド：local_thread5、試行数：38、今回のステップ:31、平均ステップ：41.4
スレッド：local_thread8、試行数：40、今回のステップ:80、平均ステップ：59.9
スレッド：local_thread6、試行数：43、今回のステップ:31、平均ステップ：48.8
スレッド：local_thread8、試行数：41、今回のステップ:30、平均ステップ：56.7
スレッド：local_thread7、試行数：46、今回のステップ:61、平均ステップ：44.7
スレッド：local_thread5、試行数：39、今回のステップ:41、平均ステップ：40.3
スレッド：local_thread1、試行数：46、今回のステップ:117、平均ステップ：44.7
スレッド：local_thread3、試行数：42、今回のステップ:52、平均ステップ：34.8
スレッド：local_thread1、試行数：47、今回のステップ:18、平均ステップ：41.5
スレッド：local_thread8、試行数：42、今回のステップ:32、平均ステップ：56.0
スレッド：local_thread3、試行数：43、今回のステップ:21、平均ステップ：34.3
スレッド：local_thread4、試行数：41、今回のステップ:137、平均ステップ：56.8
スレッド：local_thread8、試行数：43、今回のステップ:13、平均ステップ：54.1
スレッド：local_thread2、試行数：41、今回のステップ:116、平均ステップ：46.5
スレッド：local_thread6、試行数：44、今回のステップ:104、平均ステップ：58.0
スレッド：local_thre

スレッド：local_thread7、試行数：64、今回のステップ:148、平均ステップ：151.0
スレッド：local_thread8、試行数：61、今回のステップ:37、平均ステップ：115.7
スレッド：local_thread3、試行数：63、今回のステップ:147、平均ステップ：119.8
スレッド：local_thread4、試行数：61、今回のステップ:169、平均ステップ：118.4
スレッド：local_thread5、試行数：60、今回のステップ:128、平均ステップ：141.7
スレッド：local_thread6、試行数：62、今回のステップ:161、平均ステップ：138.7
スレッド：local_thread2、試行数：59、今回のステップ:192、平均ステップ：152.3
スレッド：local_thread8、試行数：62、今回のステップ:153、平均ステップ：127.5
スレッド：local_thread1、試行数：68、今回のステップ:200、平均ステップ：136.2
スレッド：local_thread4、試行数：62、今回のステップ:149、平均ステップ：131.5
スレッド：local_thread7、試行数：65、今回のステップ:191、平均ステップ：150.1
スレッド：local_thread3、試行数：64、今回のステップ:170、平均ステップ：134.6
スレッド：local_thread2、試行数：60、今回のステップ:109、平均ステップ：151.3
スレッド：local_thread1、試行数：69、今回のステップ:83、平均ステップ：130.3
スレッド：local_thread5、試行数：61、今回のステップ:184、平均ステップ：140.1スレッド：local_thread8、試行数：63、今回のステップ:116、平均ステップ：120.2

スレッド：local_thread6、試行数：63、今回のステップ:190、平均ステップ：138.7
スレッド：local_thread7、試行数：66、今回のステップ:98、平均ステップ：139.9
スレッド：local_thread4、試行数：63、今回のステップ:118、平均ステップ：127.8
スレッド：local_thread6、試行数：64、今回のステップ:

スレッド：local_thread8、試行数：82、今回のステップ:198、平均ステップ：175.2
スレッド：local_thread7、試行数：84、今回のステップ:171、平均ステップ：157.0スレッド：local_thread4、試行数：83、今回のステップ:194、平均ステップ：149.4

スレッド：local_thread6、試行数：82、今回のステップ:181、平均ステップ：153.0
スレッド：local_thread3、試行数：85、今回のステップ:87、平均ステップ：182.2
スレッド：local_thread2、試行数：79、今回のステップ:200、平均ステップ：179.4
スレッド：local_thread5、試行数：80、今回のステップ:200、平均ステップ：185.0
スレッド：local_thread8、試行数：83、今回のステップ:200、平均ステップ：175.2
スレッド：local_thread1、試行数：88、今回のステップ:200、平均ステップ：187.9
スレッド：local_thread6、試行数：83、今回のステップ:200、平均ステップ：157.6
スレッド：local_thread4、試行数：84、今回のステップ:200、平均ステップ：161.6
スレッド：local_thread7、試行数：85、今回のステップ:200、平均ステップ：157.0
スレッド：local_thread3、試行数：86、今回のステップ:200、平均ステップ：182.2
スレッド：local_thread2、試行数：80、今回のステップ:200、平均ステップ：179.4
スレッド：local_thread5、試行数：81、今回のステップ:200、平均ステップ：185.2
スレッド：local_thread8、試行数：84、今回のステップ:200、平均ステップ：175.2
スレッド：local_thread1、試行数：89、今回のステップ:197、平均ステップ：188.0
スレッド：local_thread4、試行数：85、今回のステップ:200、平均ステップ：172.4
スレッド：local_thread7、試行数：86、今回のステップ:200、平均ステップ：173.5
スレッド：local_thread6、試行数：84、今回のステッ

スレッド：local_thread4、試行数：106、今回のステップ:198、平均ステップ：94.4
スレッド：local_thread1、試行数：109、今回のステップ:151、平均ステップ：112.4
スレッド：local_thread7、試行数：102、今回のステップ:160、平均ステップ：186.1
スレッド：local_thread5、試行数：99、今回のステップ:186、平均ステップ：176.6
スレッド：local_thread1、試行数：110、今回のステップ:41、平均ステップ：105.2
スレッド：local_thread3、試行数：105、今回のステップ:163、平均ステップ：163.1
スレッド：local_thread8、試行数：103、今回のステップ:127、平均ステップ：138.1
スレッド：local_thread2、試行数：99、今回のステップ:133、平均ステップ：146.5
スレッド：local_thread6、試行数：102、今回のステップ:174、平均ステップ：188.8
スレッド：local_thread4、試行数：107、今回のステップ:108、平均ステップ：103.0
スレッド：local_thread1、試行数：111、今回のステップ:130、平均ステップ：104.7
スレッド：local_thread7、試行数：103、今回のステップ:146、平均ステップ：180.7
スレッド：local_thread5、試行数：100、今回のステップ:136、平均ステップ：170.2
スレッド：local_thread6、試行数：103、今回のステップ:138、平均ステップ：182.6
スレッド：local_thread3、試行数：106、今回のステップ:162、平均ステップ：159.3
スレッド：local_thread2、試行数：100、今回のステップ:160、平均ステップ：142.5
スレッド：local_thread8、試行数：104、今回のステップ:185、平均ステップ：136.6
スレッド：local_thread3、試行数：107、今回のステップ:25、平均ステップ：141.8
スレッド：local_thread4、試行数：108、今回のステップ:187、平均ステップ：115.9
スレッド：local_thread

スレッド：local_thread5、試行数：120、今回のステップ:168、平均ステップ：47.6
スレッド：local_thread3、試行数：124、今回のステップ:200、平均ステップ：144.3
スレッド：local_thread6、試行数：123、今回のステップ:200、平均ステップ：73.7
スレッド：local_thread2、試行数：118、今回のステップ:147、平均ステップ：84.8
スレッド：local_thread1、試行数：128、今回のステップ:175、平均ステップ：116.7
スレッド：local_thread7、試行数：123、今回のステップ:195、平均ステップ：56.7
スレッド：local_thread8、試行数：126、今回のステップ:200、平均ステップ：69.5
スレッド：local_thread4、試行数：123、今回のステップ:186、平均ステップ：151.0
スレッド：local_thread5、試行数：121、今回のステップ:192、平均ステップ：65.7
スレッド：local_thread6、試行数：124、今回のステップ:170、平均ステップ：70.7
スレッド：local_thread3、試行数：125、今回のステップ:180、平均ステップ：142.3
スレッド：local_thread2、試行数：119、今回のステップ:200、平均ステップ：84.8
スレッド：local_thread7、試行数：124、今回のステップ:175、平均ステップ：73.0
スレッド：local_thread1、試行数：129、今回のステップ:191、平均ステップ：115.8
スレッド：local_thread8、試行数：127、今回のステップ:173、平均ステップ：74.7
スレッド：local_thread4、試行数：124、今回のステップ:187、平均ステップ：149.7
スレッド：local_thread5、試行数：122、今回のステップ:161、平均ステップ：80.9
スレッド：local_thread6、試行数：125、今回のステップ:200、平均ステップ：86.1
スレッド：local_thread3、試行数：126、今回のステップ:200、平均ステップ：142.3
スレッド：local_thread2、試行数：1

Exception in thread Thread-12:
Traceback (most recent call last):
  File "/usr/lib/python3.6/threading.py", line 916, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.6/threading.py", line 864, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-8-fcfd38cd7e89>", line 25, in <lambda>
    job = lambda: worker.run()      # この辺は、マルチスレッドを走らせる作法だと思って良い
  File "<ipython-input-7-541144f10adc>", line 21, in run
    self.environment.run()
  File "<ipython-input-6-01dc5680a35c>", line 21, in run
    s = self.env.reset()
  File "/usr/local/lib/python3.5/dist-packages/gym/wrappers/monitor.py", line 39, in reset
    self._after_reset(observation)
  File "/usr/local/lib/python3.5/dist-packages/gym/wrappers/monitor.py", line 193, in _after_reset
    self.reset_video_recorder()
  File "/usr/local/lib/python3.5/dist-packages/gym/wrappers/monitor.py", line 214, in reset_video_recorder
    self.video_recorder.capture_frame()
  File "/usr/local/lib/python3.5/dist-package